# Prompts Templating for OpenAI - LangChain


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "langchain-course"

## Basic Prompting
We'll start by looking at the various parts of our prompt. For RAG use-cases we'll typically have three core components however this is very use-cases dependant and can vary significantly. Nonetheless, for RAG we will typically see:

- **Rules for our LLM:** this part of the prompt sets up the behavior of our LLM, how it should approach responding to user queries, and simply providing as much information as possible about what we're wanting to do as possible. We typically place this within the system prompt of an chat LLM.

- **Context:** this part is RAG-specific. The context refers to some external information that we may have retrieved from a web search, database query, or often a vector database. This external information is the Retrieval Augmentation part of RAG. For chat LLMs we'll typically place this inside the chat messages between the assistant and user.

- **Question:** this is the input from our user. In the vast majority of cases the question/query/user input will always be provided to the LLM (and typically through a user message). However, the format and location of this being provided often changes.

- **Answer:** this is the answer from our assistant, again this is very typical and we'd expect this with every use-case.

The below is an example of how a RAG prompt may look:

- **(Rules) For Our Prompt:**
Answer the question based on the context below,                 
if you cannot answer the question using the                     
provided information answer with "I don't know"                 

- **Context AI has:**
Context: Aurelio AI is an AI development studio                 
focused on the fields of Natural Language Processing (NLP)      
and information retrieval using modern tooling                  
such as Large Language Models (LLMs),                           
vector databases, and LangChain.                                

- **User Question:**
Question: Does Aurelio AI do anything related to LangChain?

- **AI Answer:**
Answer:                                                         


In [2]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the provided information answer with "I don't know".

Context: {context}
"""

In [4]:
# LangChain uses a ChatPromptTemplate object to format the various prompt types into a single list which will 
# be passed to our LLM:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])

When we call the template it will expect us to provide two variables, the context and the query. Both of these variables are pulled from the strings we wrote, as LangChain interprets curly-bracket syntax (ie {context} and {query}) as indicating a dynamic variable that we expect to be inserted at query time. We can see that these variables have been picked up by our template object by viewing it's input_variables attribute:

In [7]:
prompt_template.input_variables

['context', 'query']

In [8]:
# We can also view the structure of the messages (currently prompt templates) that the ChatPromptTemplate will construct by viewing the messages attribute:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the provided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

From this, we can see that each tuple provided when using ChatPromptTemplate.from_messages becomes an individual prompt template itself. Within each of these tuples, the first value defines the role of the message, which is typically system, human, or ai. Using these tuples is shorthand for the following, more explicit code:

In [9]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [10]:
# We can see the structure of this new chat prompt template is identical to our previous:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the provided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

## Invoking our LLM with Templates

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [13]:
pipeline = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"],
    }
    | prompt_template
    | llm
)

In [14]:
# Now let's define a query and some relevant context and invoke our pipeline.
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

pipeline.invoke({"query": query, "context": context})

AIMessage(content='Aurelio AI is an AI company that develops tooling for AI engineers, focusing on language AI. They have expertise in building AI agents and information retrieval. The company is known for several open source frameworks, including Semantic Router and Semantic Chunkers, and offers an AI Platform that provides engineers with tools to build with AI. Additionally, they provide development services to help other organizations bring their AI technology to market.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 182, 'total_tokens': 264, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_86d0290411', 'id': 'chatcmpl-BGmkgXARLOwwsGEX2WEU9JPxOU4CC', 'finish_reason': 'stop', 'logprobs': None}, id

Our LLM pipeline is able to consume the information from the context and use it to answer the user's query. Ofcourse, we would not usually be feeding in both a question and an answer into an LLM manually. Typically, the context would be retrieved from a vector database, via web search, or from elsewhere. We will cover this use-case in full and build a functional RAG pipeline in a future chapter.

## Few Shot Prompting

Many State-of-the-Art (SotA) LLMs are incredible at instruction following. Meaning that it requires much less effort to get the intended output or behavior from these models than is the case for older LLMs and smaller LLMs.

Before creating an example let's first see how to use LangChain's few shot prompting objects. We will provide multiple examples and we'll feed them in as sequential human and ai messages so we setup the template like this:

In [ ]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

# Then we define a list of examples with dictionaries containing the correct input and output keys.
examples = [
    {"input": "Here is query #1", "output": "Here is answer #1"},
    {"input": "Here is query #2", "output": "Here is answer #2"},
    {"input": "Here is query #3", "output": "Here is answer #3"},
]